In [16]:
from os import environ
from decouple import config, AutoConfig
from bs4 import BeautifulSoup
import re
import psycopg2

config = AutoConfig(search_path=environ["HOME"])  # look for ~/.env
# dbConnectionString = config("mcp")
dbConnectionString = config("psyco_mcp")
conn = psycopg2.connect(dbConnectionString)

char_file = "chars.html"
tactics_file = "tactics.html"
goons_file = "goons.html"


In [17]:
with open(char_file) as fp_chars:
    soup_chars = BeautifulSoup(fp_chars, "html.parser")
with open(tactics_file) as fp_tactics:
    soup_tactics = BeautifulSoup(fp_tactics, "html.parser")
with open(goons_file) as fp_goons:
    soup_goons = BeautifulSoup(fp_goons, "html.parser")


In [18]:
characters = []
char_affil_list = []
for li in soup_chars.find_all("li", class_="category"):
    if li.h3.text.strip() == "Characters":
        for rs in li.find_all("li", class_="rootselection"):
            affiliations = []
            h4 = rs.h4.text
            fullname = str(re.findall(r"(.+) \[", h4)[0]).replace(r" {BETA}", "")
            heroname = re.findall(r"(.+) \(", h4)[0]
            secret_id = re.findall(r"\((.+)\)", h4)[0]
            threat_lvl = re.findall(r"\[(\d+) ", h4)[0]

            for p in rs.find_all("p", class_="category-names"):
                for span in p.find_all("span", class_="caps"):
                    categories = span.text
                    for cat in categories.split(","):
                        if not "Character" in cat:
                            char_affil_list.append([fullname, cat.strip()])

            characters.append(
                {
                    "type": "Character",
                    "Heroname": heroname,
                    "SecretID": secret_id,
                    "full_name": fullname,
                    "threat": int(threat_lvl),
                    "affiliations": affiliations,
                }
            )


In [19]:
goons = []
for li in soup_goons.find_all("li", class_="category"):
    if li.h3.text.strip() == "Grunts":
        for sub_li in li.find_all("li", class_="rootselection"):
            friends = []
            goon_string = sub_li.h4.text.strip()
            char_type = "Grunt"
            threat = 0
            heroname = re.findall(r"(.+) \[", goon_string)[0]
            for i in str(re.findall(r"\[(.+)\]", goon_string)[0]).split(","):
                friend = i.strip()
                goons.append(
                    {
                        "type": char_type,
                        "heroname": heroname,
                        "threat": int(threat),
                        "friend": friend,
                    }
                )


In [20]:
affiliations = []
for p in soup_chars.find_all("p", class_="category-names"):
    for span in p.find_all("span", class_="caps"):
        categories = span.text
        for cat in categories.split(","):
            if not "Character" in cat:
                if cat.strip() not in affiliations:
                    affiliations.append(cat.strip())
affiliations.sort()


In [21]:
tactics_cards = []
for table in soup_tactics.find_all("table"):
    for tr in table.find_all("tr"):
        individual_card = []
        name = ""
        affiliation = ""
        type_ = ""
        text_ = ""
        generic = 0
        count = 0
        for td in tr.find_all("td"):
            if td.has_attr("class"):
                name = td.text.strip()
                count += 1
            elif count == 1:
                affiliation = td.text.strip()
                count += 1
            elif count == 2:
                type_ = td.text.strip()
                count += 1
            else:
                text_ = td.text.strip()
        if name != "":
            tactics_cards.append(
                {
                    "name": name,
                    "affiliation": affiliation,
                    "type": type_,
                    "cardtext": text_,
                }
            )


In [22]:
char_tactics = []
for i in characters:
    cname = i["full_name"]
    lower_name = i["Heroname"].lower()
    affil = i["affiliations"]
    for t in tactics_cards:
        ttext = t["cardtext"].lower()
        taffil = t["affiliation"]
        if lower_name in ttext or taffil == affil:
            char_tactics.append({"Character": cname, "Card": t["name"]})


In [23]:
x = 1
if x == 1:
    upsert_chars = """
            insert into chars (type, heroname,secretid,full_name,threat) values (
                %s, %s, %s, %s, %s
                )
                on conflict (full_name)
                do update set
                    (type,secretid,heroname,threat)
                    = (excluded.type, excluded.secretid, excluded.heroname, excluded.threat)
            """
    with conn.cursor() as cur:
        for i in characters:
            cur.execute(
                upsert_chars,
                (i["type"], i["Heroname"], i["SecretID"], i["full_name"], i["threat"]),
            )

    conn.commit()


In [24]:
x = 1
if x == 1:
    upsert_goons = """
            insert into chars (type, heroname,secretid,full_name,threat) values (
                %s, %s, %s, %s, %s
                )
                on conflict (full_name)
                do update set
                    (type,secretid,heroname,threat)
                    = (excluded.type, excluded.secretid, excluded.heroname, excluded.threat)
            """

    
    with conn.cursor() as cur:
        for i in goons:
            cur.execute(
                upsert_goons,
                (i["type"], i["heroname"], i["heroname"], i["heroname"], i["threat"]),
            )

    conn.commit()


In [25]:
x = 1
if x == 1:
    upsert_affiliations = """
            insert into affiliations (name) values (%s)
            on conflict (name)
            do nothing
            """

    with conn.cursor() as cur:
        for i in affiliations:
            cur.execute(
                upsert_affiliations,
                (i,),
            )

    conn.commit()


In [26]:
x = 1
if x == 1:
    upsert_char_affil_list = """
            insert into char_affil(AffiliationID,CharacterID)
            select
            a.affiliationid as AffiliationID
            ,(select c.characterid as CharacterID from chars c where c.full_name like %s)
            from affiliations a where a.name like %s
            on conflict on constraint char_affil_pkey
            do nothing
            """
    
    with conn.cursor() as cur:
        for i in char_affil_list:
            cur.execute(
                upsert_char_affil_list,
                (
                    i[0],
                    i[1],
                ),
            )

    conn.commit()


In [27]:
x = 1
if x == 1:
    upsert_goon_affil = """
            insert into char_affil(AffiliationID,CharacterID)
            select
            ca.affiliationid as AffiliationID
            ,(select c.characterid as CharacterID from chars c where c.full_name like %s)
            from char_affil ca
            join chars c on c.characterid = ca.characterid
            where position(%s in c.full_name)>0 --this is gross
            on conflict on constraint char_affil_pkey
            do nothing
    """
    
    with conn.cursor() as cur:
        for i in goons:
            cur.execute(
                upsert_goon_affil,
                (
                    i["heroname"],
                    i["friend"],
                ),
            )
    conn.commit()


In [28]:
x = 1
if x == 1:
    upsert_tactics = """
            insert into tactics(name,affiliationid,type,cardtext)
            select
            %s as name
            ,affiliationid
            ,%s as type
            ,%s as cardtext
            from affiliations
            where name like %s
            on conflict on constraint tactics_name_key
            do nothing
            """
    
    with conn.cursor() as cur:
        for i in tactics_cards:
            cur.execute(
                upsert_tactics,
                (
                    i["name"],
                    i["type"],
                    i["cardtext"],
                    i["affiliation"],
                ),
            )
    conn.commit()


In [29]:
x = 1
if x == 1:
    upsert_char_tactics_list = """
            insert into char_tactics(characterid,tacticsid)
            select
            c.characterid
            ,(select tacticsid from tactics where name like %s)
            from chars c
            where c.full_name like %s
            on conflict on constraint char_tactics_pkey
            do nothing
            """
    
    with conn.cursor() as cur:
        for i in char_tactics:
            cur.execute(
                upsert_char_tactics_list,
                (
                    i["Card"],
                    i["Character"],
                ),
            )
    conn.commit()
